In [1]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [2]:
DATASET_NAME = 'Amazon-Google'

In [3]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [4]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

[array([[-0.12755906,  0.20850394, -0.18645669, -0.03307087, -0.02834646,
        -0.00125984,  0.03023622,  0.23244094, -0.06488189,  0.08724409],
       [-0.14312153, -0.03562616, -0.10934608,  0.15376311,  0.0442628 ,
        -0.08482418,  0.04333745,  0.12152992, -0.15175817, -0.1124306 ],
       [-0.04335466, -0.03861644, -0.08268183,  0.21013978, -0.05875385,
         0.17910448, -0.01066098, -0.13219616,  0.01445155, -0.23004027],
       [ 0.12727585, -0.13655101, -0.03211955,  0.09498454, -0.06664377,
         0.07488835, -0.0223291 ,  0.14943318,  0.1580213 , -0.13775335],
       [ 0.10845516, -0.13275385,  0.17483208, -0.04583169, -0.06440142,
        -0.17641249, -0.01639668,  0.15428684,  0.01916239, -0.1074674 ]]), array([[-0.21074933,  0.07136485, -0.02988403, -0.15789474, -0.1382694 ,
        -0.02520071, -0.08318466, -0.12466548, -0.03925067, -0.11953613],
       [ 0.01567091, -0.02840353, -0.13883448,  0.09402547, -0.13761019,
        -0.16062684, -0.23922625,  0.08006

In [5]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


1
32


In [6]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [7]:
## Open dataset 
df = pd.read_csv('../../lsh-test-data/' + DATASET_NAME + '-embeddings.csv')

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 150)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

K: 1, L: 1, Precision:0.5769230769230769
K: 1, L: 2, Precision:0.9358974358974359
K: 1, L: 3, Precision:0.9957264957264957
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.4188034188034188
K: 2, L: 2, Precision:0.7948717948717948
K: 2, L: 3, Precision:0.9700854700854701
K: 2, L: 4, Precision:0.9957264957264957
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.2948717948717949
K: 3, L: 2, Precision:0.6239316239316239
K: 3, L: 3, Precision:0.8974358974358975
K: 3, L: 4, Precision:0.9572649572649573
K: 3, L: 5, Precision:0.9829059829059829
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8, Precision:1.0
K: 3, L: 9, Precision:1.0
K: 3, L: 10, Precision:1.0
K: 4, L: 1, Precision:0.2094017094017

K: 21, L: 1, Precision:0.0
K: 21, L: 2, Precision:0.004273504273504274
K: 21, L: 3, Precision:0.008547008547008548
K: 21, L: 4, Precision:0.008547008547008548
K: 21, L: 5, Precision:0.004273504273504274
K: 21, L: 6, Precision:0.02564102564102564
K: 21, L: 7, Precision:0.017094017094017096
K: 21, L: 8, Precision:0.02564102564102564
K: 21, L: 9, Precision:0.021367521367521368
K: 21, L: 10, Precision:0.042735042735042736
K: 22, L: 1, Precision:0.004273504273504274
K: 22, L: 2, Precision:0.008547008547008548
K: 22, L: 3, Precision:0.0
K: 22, L: 4, Precision:0.017094017094017096
K: 22, L: 5, Precision:0.01282051282051282
K: 22, L: 6, Precision:0.021367521367521368
K: 22, L: 7, Precision:0.02564102564102564
K: 22, L: 8, Precision:0.01282051282051282
K: 22, L: 9, Precision:0.03418803418803419
K: 22, L: 10, Precision:0.038461538461538464
K: 23, L: 1, Precision:0.0
K: 23, L: 2, Precision:0.004273504273504274
K: 23, L: 3, Precision:0.004273504273504274
K: 23, L: 4, Precision:0.008547008547008548